In [1]:
"""
The MIT License (MIT)

Copyright (c) 2016 Christian August Reksten-Monsen

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

#visibility graph

from collections import defaultdict
import numpy as np
import math
import heapq

class Point(object):
    __slots__ = ('x', 'y', 'polygon_id')

    def __init__(self, x, y, polygon_id=-1):
        self.x = float(x)
        self.y = float(y)
        self.polygon_id = polygon_id

    def __eq__(self, point):
        return point and self.x == point.x and self.y == point.y

    def __ne__(self, point):
        return not self.__eq__(point)

    def __lt__(self, point):
        """ This is only needed for shortest path calculations where heapq is
            used. When there are two points of equal distance, heapq will
            instead evaluate the Points, which doesnt work in Python 3 and
            throw a TypeError."""
        return hash(self) < hash(point)

    def __str__(self):
        return "(%.2f, %.2f)" % (self.x, self.y)

    def __hash__(self):
        return self.x.__hash__() ^ self.y.__hash__()

    def __repr__(self):
        return "Point(%.2f, %.2f)" % (self.x, self.y)

class Edge(object):
    __slots__ = ('p1', 'p2')

    def __init__(self, point1, point2):
        self.p1 = point1
        self.p2 = point2

    def get_adjacent(self, point):
        if point == self.p1:
            return self.p2
        return self.p1

    def __contains__(self, point):
        return self.p1 == point or self.p2 == point

    def __eq__(self, edge):
        if self.p1 == edge.p1 and self.p2 == edge.p2:
            return True
        if self.p1 == edge.p2 and self.p2 == edge.p1:
            return True
        return False

    def __ne__(self, edge):
        return not self.__eq__(edge)

    def __str__(self):
        return "({}, {})".format(self.p1, self.p2)

    def __repr__(self):
        return "Edge({!r}, {!r})".format(self.p1, self.p2)

    def __hash__(self):
        return self.p1.__hash__() ^ self.p2.__hash__()

class Graph(object):
    """
    A Graph is represented by a dict where the keys are Points in the Graph
    and the dict values are sets containing Edges incident on each Point.
    A separate set *edges* contains all Edges in the graph.

    The input must be a list of polygons, where each polygon is a list of
    in-order (clockwise or counter clockwise) Points. If only one polygon,
    it must still be a list in a list, i.e. [[Point(0,0), Point(2,0),
    Point(2,1)]].

    *polygons* dictionary: key is a integer polygon ID and values are the
    edges that make up the polygon. Note only polygons with 3 or more Points
    will be classified as a polygon. Non-polygons like just one Point will be
    given a polygon ID of -1 and not maintained in the dict.
    """

    def __init__(self, polygons):
        self.graph = defaultdict(set)
        self.edges = set()
        self.polygons = defaultdict(set)
        pid = 0
        for polygon in polygons:
            if polygon[0] == polygon[-1] and len(polygon) > 1:
                polygon.pop()
            for i, point in enumerate(polygon):
                sibling_point = polygon[(i + 1) % len(polygon)]
                edge = Edge(point, sibling_point)
                if len(polygon) > 2:
                    point.polygon_id = pid
                    sibling_point.polygon_id = pid
                    self.polygons[pid].add(edge)
                self.add_edge(edge)
            if len(polygon) > 2:
                pid += 1

    def get_adjacent_points(self, point):
        return [edge.get_adjacent(point) for edge in self[point]]

    def get_points(self):
        return list(self.graph)

    def get_edges(self):
        return self.edges
    
    def get_polygons(self):
        return self.polygons

    def add_edge(self, edge):
        self.graph[edge.p1].add(edge)
        self.graph[edge.p2].add(edge)
        self.edges.add(edge)

    def __contains__(self, item):
        if isinstance(item, Point):
            return item in self.graph
        if isinstance(item, Edge):
            return item in self.edges
        return False

    def __getitem__(self, point):
        if point in self.graph:
            return self.graph[point]
        return set()

    def __str__(self):
        res = ""
        for point in self.graph:
            res += "\n" + str(point) + ": "
            for edge in self.graph[point]:
                res += str(edge)
        return res

    def __repr__(self):
        return self.__str__()

In [2]:
# 0 - collinear
# 1 - clockwise
# -1 - counterclockwise

def orientation(p1, p2, p3):
    angle = (float)(p2.y - p1.y)*(p3.x-p2.x) - (float)(p3.y-p2.y)*(p2.x-p1.x)
    if(angle < 0):
        return -1
    if angle > 0:
        return 1
    return 0

In [3]:
def onSegment(p, q, r): 
    if ( (q.x <= max(p.x, r.x)) and (q.x >= min(p.x, r.x)) and 
           (q.y <= max(p.y, r.y)) and (q.y >= min(p.y, r.y))): 
        return True
    return False

In [4]:
def intersect(edge, obstacle):
    if(obstacle.p1 in edge) or (obstacle.p2 in edge):
        return False
    o1 = orientation(edge.p1, edge.p2, obstacle.p1)
    o2 = orientation(edge.p1, edge.p2, obstacle.p2)
    o3 = orientation(obstacle.p1, obstacle.p2, edge.p1)
    o4 = orientation(obstacle.p1, obstacle.p2, edge.p2)
    #general case 
    if (o1 != o2) and (o3 != o4):
        return True
    if (o1 == 0) and onSegment(edge.p1, obstacle.p1, edge.p2):
        return True
    if (o2 == 0) and onSegment(edge.p1, obstacle.p2, edge.p2):
        return True
    if (o3 == 0) and onSegment(obstacle.p1, obstacle.p2, edge.p1):
        return True
    if (o4 == 0) and onSegment(obstacle.p1, obstacle.p2, edge.p2):
        return True
    return False;

In [5]:
#naive visibility graph
def build_graph(graph, start = None, goal = None):
    obstacles = []
    for polygons in graph.get_polygons().values():
        obstacles += polygons
    points = graph.get_points()
    if start:
        points.append(start)
    if goal:
        points.append(goal)
    for point in points:
        for other_point in graph.get_points():
            if point == other_point: #looping over the other n-1 vertcies, assuming that n is the total number of vertices
                continue
            edge = Edge(point, other_point)
            if edge in graph: #if it is already part of the graph, it's either an edge of the obstacle, since it s enlarged we can traverse it, and if it's not it was obtained when checking the edge when the roles of points were reversed    
                continue
            intersects = False
            for obstacle in obstacles:
                if(intersect(edge, obstacle)):
                    intersects = True
                    break
            if intersects == False:
                graph.add_edge(edge) 
    return graph

In [6]:
def distanceBetweenPoints(p1, p2):
    return math.sqrt((p2.x - p1.x)**2 + (p2.y - p1.y)**2)
def heuristic(point, goal):
    return distanceBetweenPoints(point, goal)

In [7]:
def Astar(polys, start, goal):
    graph = Graph(polys)
    #check if start and goal are valid points !!
    build_graph(graph, start, goal)
    print("Graph built")
    points = graph.get_points()
    opened = [(heuristic(start,goal),start)]
    heapq.heapify(opened)
    closed = []
    cameFrom = dict()
    h = dict()
    costs = dict(zip(points, [np.inf for x in range(len(points))]))
    costs[start] = 0
    cameFrom = dict()
    nodes = []
    while len(opened) != 0:
        currentEstimate, current = heapq.heappop(opened)
        closed.append(current)
        if current == goal:
            while current != start:
                nodes.append(current)
                temp = cameFrom[current]
                current = temp
            nodes.append(current)
            return nodes, closed
        for neighbour in graph.get_adjacent_points(current):
            cost = costs[current] + distanceBetweenPoints(current, neighbour)
            if neighbour in costs:
                if cost >= costs[neighbour]:
                    continue
            opened.append((cost + heuristic(neighbour, goal), neighbour))
            cameFrom[neighbour] = current
            costs[neighbour] = cost           
    print("No path found to goal")
    return nodes, closed
        
polys = [[Point(0.0,1.0), Point(3.0,1.0), Point(1.5,4.0)], [Point(4.0,4.0), Point(7.0,4.0), Point(5.5,8.0)]]
start = Point(0.0,0.0)
goal = Point(9.0, 10.0)

nodes, closed = Astar(polys, start, goal)
nodes = nodes[::-1]
for node in nodes:
    print(node)

Graph built
(0.00, 1.00)
(1.50, 4.00)
(5.50, 8.00)
(9.00, 10.00)


In [17]:
points = np.zeros((len(nodes),2))
print(points)
for i in range(len(nodes)):
    points[i] = np.array([nodes[i].x, nodes[i].y])
print(points.T)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[[ 0.   1.5  5.5  9. ]
 [ 1.   4.   8.  10. ]]
